In [2]:
from VQVAE import VQVAE
import time
import jax
import jax.numpy as jnp
import torch

# import wandb
from dvae import DiscreteVAE

dvae = DiscreteVAE(
            channels=80,
            normalization=None,
            positional_dims=1,
            num_tokens=1024,
            codebook_dim=512,
            hidden_dim=512,
            num_resnet_blocks=3,
            kernel_size=3,
            num_layers=2,
            use_transposed_convs=False,
        )

dvae.cuda()

def to_cuda(x: torch.Tensor) -> torch.Tensor:
    if x is None:
        return None
    if torch.is_tensor(x):
        x = x.contiguous()
        if torch.cuda.is_available():
            x = x.cuda(non_blocking=True)
    return x

jax_dvae = VQVAE(jax.random.key(1))

In [3]:



batch = jnp.zeros((512, 80, 1024))
jax.vmap(jax_dvae)(batch)
print("Done first blank run")
val = jax.vmap(jax_dvae)(batch)
print("Done first blank run")


batch = jnp.ones((512, 80, 1024))
start = time.perf_counter()
val = jax.vmap(jax_dvae)(batch)
end = time.perf_counter()
print(f"Execution time: {end - start:.6f} seconds")
with torch.no_grad():
    dvae.eval()
    
    batch = torch.zeros((512, 80, 1024)).cuda()
    # start = time.perf_counter()
    dvae(batch)
    end = time.perf_counter()
    batch = torch.ones((512, 80, 1024)).cuda()
    start = time.perf_counter()
    dvae(batch)
    end = time.perf_counter()
    print(f"Execution time: {end - start:.6f} seconds")


print(val)

Done first blank run
Done first blank run
Execution time: 0.002563 seconds
Execution time: 0.004505 seconds
(Array([[[-0.03863255, -0.02389665, -0.02858979, ..., -0.02428682,
         -0.03213526, -0.04179664],
        [-0.00363815, -0.01193823, -0.00896992, ..., -0.008536  ,
         -0.00655342, -0.00100758],
        [-0.05662166, -0.04697611, -0.04690373, ..., -0.04900101,
         -0.05542943, -0.06173106],
        ...,
        [-0.00637812, -0.01877609, -0.01943821, ..., -0.01428112,
         -0.01442412, -0.00423253],
        [-0.10770497, -0.09449294, -0.09988593, ..., -0.09684483,
         -0.10558368, -0.09420164],
        [-0.00521094, -0.13641626, -0.12592846, ..., -0.12530902,
         -0.11837769, -0.11257334]],

       [[-0.03863255, -0.02389665, -0.02858979, ..., -0.02428682,
         -0.03213526, -0.04179664],
        [-0.00363815, -0.01193823, -0.00896992, ..., -0.008536  ,
         -0.00655342, -0.00100758],
        [-0.05662166, -0.04697611, -0.04690373, ..., -0.0490